# Using a pre-trained model(VGG16) in Keras
1. Classify ImageNet classes
1. Extract the feature of a given image  

# Reference
* [Classify ImageNet classes with ResNet50](https://keras.io/applications/#classify-imagenet-classes-with-resnet50)
* [Extract features with VGG16](https://keras.io/applications/#extract-features-with-vgg16)
* [preprocess_input() method in keras](https://stackoverflow.com/questions/47555829/preprocess-input-method-in-keras)
* [Keras-VGG16](https://keras.io/applications/#vgg16)  
  * include_top: whether to include the 3 fully-connected layers at the top of the network.

# DataSet
* [Dogs vs. Cats](https://www.kaggle.com/c/dogs-vs-cats/overview)

In [1]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions
import keras
import numpy as np
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

Using TensorFlow backend.


In [2]:
# Environment
!python --version
print("TensorFlow:", tf.__version__)
print("Keras:", keras.__version__)
!nvidia-smi | grep Version | cut -c 3-

Python 3.7.3
TensorFlow: 1.13.1
Keras: 2.2.4
NVIDIA-SMI 415.27       Driver Version: 415.27       CUDA Version: 10.0     |


In [3]:
# Workaround Issue: "Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

In [4]:
img_path = '/home/ryanyao/docker_mount/data/dogs_vs_cats/train/dog.9999.jpg'

## 1. Classify ImageNet classes

In [5]:
keras.backend.clear_session()
model = VGG16(weights='imagenet', include_top=True)
model.summary()

# Loads an image into PIL format.
img = image.load_img(img_path, target_size=(224, 224))

# Converts a PIL Image instance to a Numpy array. shape=(224, 224, 3)
img_data = image.img_to_array(img)                         
print("image.img_to_array(img):", img_data.shape)

# Insert a new axis that will appear at the `axis` position in the expanded array shape. shape=(1, 224, 224, 3)
img_data = np.expand_dims(img_data, axis=0)                
print("np.expand_dims(img_data, axis=0):", img_data.shape)

# preprocess_input() is meant to adequate your image to the format the model require. shape=(1, 224, 224, 3)
img_data = preprocess_input(img_data)
print("preprocess_input(img_data):", img_data.shape)

preds = model.predict(img_data)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
bloc

## 2. Extract the feature of a given image

In [6]:
keras.backend.clear_session()
model = VGG16(weights='imagenet', include_top=False)
model.summary()

img = image.load_img(img_path, target_size=(224, 224))
img_data = image.img_to_array(img)
img_data = np.expand_dims(img_data, axis=0)
img_data = preprocess_input(img_data)

vgg16_feature = model.predict(img_data)

print(vgg16_feature.shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________